In [1]:
from selenium import webdriver
driver = webdriver.Chrome(executable_path = 'C:/Users/Sahil/.wdm/drivers/chromedriver/win32/110.0.5481/chromedriver.exe')
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="8a9f417e6fa39dcd5d447fafc35569d1")>

In [2]:
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd

In [37]:
class ToxinPred_Toxicity:
    def __init__(self):
        self.base_url = 'https://webs.iiitd.edu.in/raghava/toxinpred2/batch.html'
        
    def get_data(self, source_filepath):
        driver.get(self.base_url)
        
        sequences = []
        toxicity_scores = []
        toxic = []
        
        chose_file = driver.find_element_by_xpath('/html/body/header/div[2]/section/form/table/tbody/tr/td/font/p/font[2]/input')
        chose_file.send_keys(source_filepath)
        
        submit = driver.find_element_by_xpath('/html/body/header/div[2]/section/form/table/tbody/tr/td/font/font/p[3]/font/font[2]/input[2]')
        submit.click()
        
        WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH, '/html/body/header/div[2]/main/div/table[2]')))
        
        seq = driver.find_elements_by_xpath('/html/body/header/div[2]/main/div/table[2]/tbody/tr/td[1]')
        score = driver.find_elements_by_xpath('/html/body/header/div[2]/main/div/table[2]/tbody/tr/td[5]')
        tox = driver.find_elements_by_xpath('/html/body/header/div[2]/main/div/table[2]/tbody/tr/td[6]')
        
        for s, st, to in zip(seq, score, tox):
            sequences.append(s.text)
            toxicity_scores.append(float(st.text))
            toxic.append(to.text)
            
        df1 = pd.DataFrame({'Sequence':sequences, 'Toxicity_score':toxicity_scores, 'Toxicity':toxic})
        return df1
    
    
    def convert_fasta_to_dataframe(self, source_filepath):
        
        with open(source_filepath, 'r') as f:
            pep = f.readlines()
        
        seq = []
        epit = []

        for i in range(0, len(pep)):
            if i%2 == 0:
                seq.append(pep[i].split('>')[1].split('\n')[0])
            else:
                epit.append(pep[i].split('\n')[0])
                
        df2 = pd.DataFrame({'Sequence':seq, 'Epitopes':epit})
        return df2
    
    
    def merge_results(self, df1, df2):
        df = pd.merge(df1, df2, on = 'Sequence')
        return df
    
    def save_results(self, df, dest_filepath):
        df.to_csv(dest_filepath, index = False)
        
        
        
        

In [39]:
source_filepath = r'C:\Users\Sahil\Omdena\Development of Vaccines for JC Virus\mhc1_epitopes_fasta.txt'
dest_filepath = r'C:\Users\Sahil\Omdena\Development of Vaccines for JC Virus\toxinpred_toxicity_results.csv'

In [38]:
tpt = ToxinPred_Toxicity()

In [40]:
data1 = tpt. get_data(source_filepath)
data1

,Sequence,Toxicity_score,Toxicity
0,seq0,0.83,Toxin
1,seq1,0.68,Toxin
2,seq2,0.86,Toxin
3,seq3,0.88,Toxin
4,seq4,0.90,Toxin
...,...,...,...
60,seq60,0.50,Non-Toxin
61,seq61,0.47,Non-Toxin
62,seq62,0.66,Toxin
63,seq63,0.85,Toxin


In [41]:
data2 = tpt.convert_fasta_to_dataframe(source_filepath)
data2

,Sequence,Epitopes
0,seq0,ACFAVYTTK
1,seq1,AVDTVLAKK
2,seq2,AWLHCLLPK
3,seq3,AYLRKCKEF
4,seq4,FLICKGVNK
...,...,...
60,seq60,SQHSTPPKK
61,seq61,TPHRHRVSA
62,seq62,VEESIQGGL
63,seq63,YCFDCFRQW


In [42]:
data = tpt.merge_results(data1, data2)
data

,Sequence,Toxicity_score,Toxicity,Epitopes
0,seq0,0.83,Toxin,ACFAVYTTK
1,seq1,0.68,Toxin,AVDTVLAKK
2,seq2,0.86,Toxin,AWLHCLLPK
3,seq3,0.88,Toxin,AYLRKCKEF
4,seq4,0.90,Toxin,FLICKGVNK
...,...,...,...,...
60,seq60,0.50,Non-Toxin,SQHSTPPKK
61,seq61,0.47,Non-Toxin,TPHRHRVSA
62,seq62,0.66,Toxin,VEESIQGGL
63,seq63,0.85,Toxin,YCFDCFRQW


In [43]:
tpt.save_results(data, dest_filepath)

In [23]:
with open('mhc1_epitopes_fasta.txt', 'r') as f:
    pep = f.readlines()
    
pep

['>seq0\n',
 'ACFAVYTTK\n',
 '>seq1\n',
 'AVDTVLAKK\n',
 '>seq2\n',
 'AWLHCLLPK\n',
 '>seq3\n',
 'AYLRKCKEF\n',
 '>seq4\n',
 'FLICKGVNK\n',
 '>seq5\n',
 'GVNKEYLLY\n',
 '>seq6\n',
 'ILYKKLMEK\n',
 '>seq7\n',
 'IVFNVPKRR\n',
 '>seq8\n',
 'PYHTIEESI\n',
 '>seq9\n',
 'QYMAGVAWL\n',
 '>seq10\n',
 'RLDSEISMY\n',
 '>seq11\n',
 'RTLACFAVY\n',
 '>seq12\n',
 'RYWLFKGPI\n',
 '>seq13\n',
 'SAINNFCQK\n',
 '>seq14\n',
 'SVKVNLEKK\n',
 '>seq15\n',
 'VFLLLGMYL\n',
 '>seq16\n',
 'VSWKLITEY\n',
 '>seq17\n',
 'WSSSEVPTY\n',
 '>seq18\n',
 'YSALTRDPY\n',
 '>seq19\n',
 'LSLMRKAYL\n',
 '>seq20\n',
 'NLPLMRKAY\n',
 '>seq21\n',
 'NLSLMRKAY\n',
 '>seq22\n',
 'APKKPKEPV\n',
 '>seq23\n',
 'APTKRKGEC\n',
 '>seq24\n',
 'GPLCKADSL\n',
 '>seq25\n',
 'NPTAQSQVM\n',
 '>seq26\n',
 'SPERKMLPC\n',
 '>seq27\n',
 'LLIKGGVEV\n',
 '>seq28\n',
 'NLRGFSLKL\n',
 '>seq29\n',
 'FLHCIVFNV\n',
 '>seq30\n',
 'FLTPHRHRV\n',
 '>seq31\n',
 'AITEVECFL\n',
 '>seq32\n',
 'LPALTSQEI\n',
 '>seq33\n',
 'ILMWEAVTL\n',
 '>seq34\n',
 'ILQSGMTLL

In [34]:
seq = []
epit = []

for i in range(0, len(pep)):
    if i%2 == 0:
        seq.append(pep[i].split('>')[1].split('\n')[0])
    else:
        epit.append(pep[i].split('\n')[0])
        


In [33]:
seq

['seq0',
 'seq1',
 'seq2',
 'seq3',
 'seq4',
 'seq5',
 'seq6',
 'seq7',
 'seq8',
 'seq9',
 'seq10',
 'seq11',
 'seq12',
 'seq13',
 'seq14',
 'seq15',
 'seq16',
 'seq17',
 'seq18',
 'seq19',
 'seq20',
 'seq21',
 'seq22',
 'seq23',
 'seq24',
 'seq25',
 'seq26',
 'seq27',
 'seq28',
 'seq29',
 'seq30',
 'seq31',
 'seq32',
 'seq33',
 'seq34',
 'seq35',
 'seq36',
 'seq37',
 'seq38',
 'seq39',
 'seq40',
 'seq41',
 'seq42',
 'seq43',
 'seq44',
 'seq45',
 'seq46',
 'seq47',
 'seq48',
 'seq49',
 'seq50',
 'seq51',
 'seq52',
 'seq53',
 'seq54',
 'seq55',
 'seq56',
 'seq57',
 'seq58',
 'seq59',
 'seq60',
 'seq61',
 'seq62',
 'seq63',
 'seq64']

In [35]:
epit

['ACFAVYTTK',
 'AVDTVLAKK',
 'AWLHCLLPK',
 'AYLRKCKEF',
 'FLICKGVNK',
 'GVNKEYLLY',
 'ILYKKLMEK',
 'IVFNVPKRR',
 'PYHTIEESI',
 'QYMAGVAWL',
 'RLDSEISMY',
 'RTLACFAVY',
 'RYWLFKGPI',
 'SAINNFCQK',
 'SVKVNLEKK',
 'VFLLLGMYL',
 'VSWKLITEY',
 'WSSSEVPTY',
 'YSALTRDPY',
 'LSLMRKAYL',
 'NLPLMRKAY',
 'NLSLMRKAY',
 'APKKPKEPV',
 'APTKRKGEC',
 'GPLCKADSL',
 'NPTAQSQVM',
 'SPERKMLPC',
 'LLIKGGVEV',
 'NLRGFSLKL',
 'FLHCIVFNV',
 'FLTPHRHRV',
 'AITEVECFL',
 'LPALTSQEI',
 'ILMWEAVTL',
 'ILQSGMTLL',
 'KLCTFSFLI',
 'KLMEKYSVT',
 'LLLIWFRPV',
 'LLMWEAVTV',
 'MLTERFNHI',
 'SITEVECFL',
 'SLMNVHSNG',
 'SMLNLHAGS',
 'VKNPYPISF',
 'SLQNSEFLL',
 'TLAAGLLDL',
 'TLACFAVYT',
 'CLLPKMDSV',
 'VIFDFLHCI',
 'VVFEDVKGT',
 'AVDTVAAKQ',
 'EPLVWIDCY',
 'FPLCPDTLY',
 'FPPNSDTLY',
 'IEESIQGGL',
 'LLIRGGVEV',
 'LLLGMYLEF',
 'LLMGMYLDF',
 'MELMDLLGL',
 'SPLVWIDCY',
 'SQHSTPPKK',
 'TPHRHRVSA',
 'VEESIQGGL',
 'YCFDCFRQW',
 'YCIDCFTQW']